In [ ]:
### Detected cat

In [ ]:
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms

# transform = transforms.ToTensor()
# trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
# testset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)



100%|██████████| 170M/170M [00:43<00:00, 3.90MB/s] 


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision import datasets
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

# Load model
model = MobileNetV2(weights="imagenet")

# CIFAR-10 dataset
trainset = datasets.CIFAR10(root="./data", train=True, download=True)
cifar10_classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def detect_cat_in_cifar10_sample(index):
    # Lấy ảnh và label
    img, label = trainset[index]
    true_label = cifar10_classes[label]

    # Convert sang numpy
    img = np.array(img)

    # Resize cho MobileNetV2
    img_resized = cv2.resize(img, (224, 224))
    x = np.expand_dims(img_resized, axis=0)
    x = preprocess_input(x)

    # Dự đoán
    preds = model.predict(x)
    decoded = decode_predictions(preds, top=3)[0]

    # Kiểm tra có mèo không
    predicted_cat = any("cat" in lbl.lower() for _, lbl, _ in decoded)

    # In kết quả
    print(f"Ground truth CIFAR-10: {true_label}")
    print("MobileNetV2 top-3:", decoded)

    # Hiển thị ảnh
    plt.imshow(np.array(trainset[index][0]))
    plt.title(f"Ground truth: {true_label}")
    plt.axis("off")
    plt.show()

    # So sánh đúng sai
    if true_label == "cat":
        if predicted_cat:
            print(" Dự đoán ĐÚNG (ảnh này là mèo và model cũng thấy mèo)")
        else:
            print(" Dự đoán SAI (ảnh này là mèo nhưng model không thấy mèo)")
    else:
        if predicted_cat:
            print(" Dự đoán SAI (ảnh này KHÔNG phải mèo nhưng model lại thấy mèo)")
        else:
            print(" Dự đoán ĐÚNG (ảnh này không phải mèo và model cũng không thấy mèo)")


for i in range (0,50):
    result = detect_cat_in_cifar10_sample(i)


In [58]:
def evaluate_accuracy(N):
    correct = 0
    total = 0
    n_samples = N
    for i in range(n_samples):
        img, label = trainset[i]
        true_label = cifar10_classes[label]

        # Convert sang numpy
        img = np.array(img)

        # Resize cho MobileNetV2
        img_resized = cv2.resize(img, (224, 224))
        x = np.expand_dims(img_resized, axis=0)
        x = preprocess_input(x)

        # Dự đoán
        preds = model.predict(x, verbose=0)
        decoded = decode_predictions(preds, top=3)[0]

        # Model có thấy mèo không?
        predicted_cat = any("cat" in lbl.lower() for _, lbl, _ in decoded)

        # Đúng khi ground truth và dự đoán trùng nhau
        if (true_label == "cat" and predicted_cat) or (true_label != "cat" and not predicted_cat):
            correct += 1
        total += 1

    acc = correct / total
    return acc



if __name__ == "__main__":
    N =10000
    acc = evaluate_accuracy(N)
    print(f"✅ Accuracy của MobileNetV2 trên {N} ảnh CIFAR-10: {acc:.2%}")

✅ Accuracy của MobileNetV2 trên 10000 ảnh CIFAR-10: 89.78%
